collect all results into a spreadsheet

In [9]:
import sys
sys.path.append("../")
import os
import numpy as np
import nibabel as nb
import pandas as pd
import matplotlib.pylab as plt
from data.nifti_dataset import resample_nifti
from data.base_dataset import _roll2center_crop
from scipy.ndimage.measurements import center_of_mass

import DeepStrain.functions_collection as ff
import CMR_HFpEF_Analysis.Image_utils as util

main_path = '/mnt/mount_zc_NAS/Deepstrain'
data_path = '/mnt/mount_zc_NAS/HFpEF/data/HFpEF_data'
raw_spreadsheet = pd.read_excel(os.path.join(data_path, 'Patient_list', 'Important_HFpEF_Patient_list_unique_patient_w_readmission_finalized.xlsx' ))

In [10]:
# define patient list:
index_range = range(0,50)
patient_list = raw_spreadsheet.iloc[index_range]
save_folder = os.path.join(main_path, 'results/result_collections')

# Spreadsheet for geometry measurements

In [12]:
result = []
for i in range(33,34):# patient_list.shape[0]):
    patient_id_num = patient_list['OurID'].iloc[i]
    patient_id = ff.XX_to_ID_00XX(patient_id_num)
   
    # load geometry then:
    geometry_folder = os.path.join(main_path, 'results/geometry', patient_id)
    
    # circular index
    circular_index = np.load(os.path.join(geometry_folder, 'circular_index_collect.npy')).reshape(-1)  # 0:10
    
    # centers
    centers = np.load(os.path.join(geometry_folder, 'centers_collect.npy'), allow_pickle=True)
    center_delta_to_base = centers[1]
    center_delta_X_to_base = centers[1][:,0].reshape(-1) # 10:20
    center_delta_Y_to_base = centers[1][:,1].reshape(-1) # 20:30
    center_delta_Euclidean_to_base = np.sqrt(np.square(center_delta_X_to_base) + np.square(center_delta_Y_to_base)) # 30:40
    
    enclosed_area = np.asarray(centers[-1]).reshape(-1) # 40:41


    # axis len
    axis = np.load(os.path.join(geometry_folder, 'axis_len_collect.npy'), allow_pickle=True)
    major_axis = axis[:-1,0].reshape(-1) #41:51
    minor_axis = axis[:-1,1].reshape(-1) #51:61

    print(major_axis, minor_axis)

    # combine these results together into a array of (num_parameters,)
    a = np.round(np.concatenate((circular_index, center_delta_X_to_base, center_delta_Y_to_base, center_delta_Euclidean_to_base, enclosed_area, major_axis, minor_axis), axis=0),3)
    a = [patient_id_num, patient_id] + a.tolist()
    
    result.append(a)

# # save the result
# name = ['circular_index', 'center_delta_X_to_base', 'center_delta_Y_to_base', 'center_delta_Euclidean_to_base', 'enclosed_area', 'major_axis', 'minor_axis']
# column_list = ['OurID', 'PatientID']
# for n in name:
#     if n[0:3] != 'enc': # 'enclosed_area' is a single value
#         for i in range(0,10):
#             column_list.append(n + '_slice_' + str(i + 1))
#     else:
#         column_list.append(n)

# df = pd.DataFrame(result, columns=column_list)
# df_info = patient_list[['OurID','Final label if count surgery at day 0', 'Final label if not count surgery at day 0', 'ZC label', 'ED', 'ES']]

# # merge df and df_info according to "OurID", df_info first then df
# df = pd.merge(df_info, df, on='OurID')
# # move patientID to the second column
# column_to_move = 'PatientID'
# target_position = 1 
# columns = df.columns.tolist()
# columns.remove(column_to_move)
# columns.insert(target_position, column_to_move)
# df = df[columns]

# df_geometry = df

# df_geometry.to_excel(os.path.join(save_folder, 'geometry_collection.xlsx'), index=False)


[36.84242727 36.98391484 39.29692942 42.87819828 42.87819828 43.12015282
 42.54371385 36.82353644 30.58448671 20.75671191] [29.39518966 31.85918063 35.27975118 36.38910384 36.38910384 36.44480099
 33.53364651 33.69668704 26.99696857 15.92183418]


# Spreadsheet for strain measurements, ED and ES

In [4]:
result = []
for i in range(0, patient_list.shape[0]):
    patient_id_num = patient_list['OurID'].iloc[i]
    patient_id = ff.XX_to_ID_00XX(patient_id_num)
   
    # load strain first:
    strain_folder = os.path.join(main_path, 'results/strain', patient_id)
    # global strain
    global_strain = np.load(os.path.join(strain_folder, 'global_strain.npy'))
    global_radial_strain = np.asarray(global_strain[0]).reshape(-1)   # 0:1
    global_circumferential_strain = np.asarray(global_strain[1]).reshape(-1) # 1:2

    # layer strain
    layer_strain = np.load(os.path.join(strain_folder, 'layer_strain.npy'))
    layer_radial_strain = layer_strain[0,:].reshape(-1) # 2:5
    layer_circumferential_strain = layer_strain[1,:].reshape(-1) # 5:8

    # polar strain
    polar_strain = np.load(os.path.join(strain_folder, 'polar_strain.npy'), allow_pickle=True)
    Err_aha = np.asarray(polar_strain[7][:-1]).reshape(-1) # 8:24
    Ecc_aha = np.asarray(polar_strain[5][:-1]).reshape(-1) # 24:40

    # WTCI
    wtci = np.load(os.path.join(strain_folder, 'wtci.npy'), allow_pickle=True)
    wtci_aha = np.asarray(wtci[4][:-1]).reshape(-1) # 40:56

    # combine these results together into a array of (num_parameters,)
    a = np.round(np.concatenate((global_radial_strain, global_circumferential_strain, layer_radial_strain, layer_circumferential_strain, Err_aha, Ecc_aha, wtci_aha), axis=0),3)
    a = [patient_id_num, patient_id] + a.tolist()
    result.append(a)


# save the result
walls = ['basal_anterior', 'basal_anteroseptal', 'basal_inferoseptal', 'basal_inferior', 'basal_inferolateral', 'basal_anterolateral', 'mid_anterior', 'mid_anteroseptal', 'mid_inferoseptal', 'mid_inferior', 'mid_inferolateral', 'mid_anterolateral', 'apical_anterior', 'apical_septal', 'apical_inferior', 'apical_lateral']
column_list = ['OurID', 'PatientID', 'Err_global', 'Ecc_global', 'Err_base', 'Err_mid', 'Err_apex', 'Ecc_base', 'Ecc_mid', 'Ecc_apex' ]
for n in ['Err', 'Ecc','WTCI']:
    for w in walls:
        column_list.append(n + '_' + w)
print(column_list, len(column_list))

df = pd.DataFrame(result, columns=column_list)
df_info = patient_list[['OurID','Final label if count surgery at day 0', 'Final label if not count surgery at day 0', 'ZC label', 'ED', 'ES', 'LVEF']]

# merge df and df_info according to "OurID", df_info first then df
df = pd.merge(df_info, df, on='OurID')
# move patientID to the second column
column_to_move = 'PatientID'
target_position = 1 
columns = df.columns.tolist()
columns.remove(column_to_move)
columns.insert(target_position, column_to_move)
df = df[columns]

df_strain = df
df_strain.to_excel(os.path.join(save_folder, 'strain_ED_ES_collection.xlsx'), index=False)


['OurID', 'PatientID', 'Err_global', 'Ecc_global', 'Err_base', 'Err_mid', 'Err_apex', 'Ecc_base', 'Ecc_mid', 'Ecc_apex', 'Err_basal_anterior', 'Err_basal_anteroseptal', 'Err_basal_inferoseptal', 'Err_basal_inferior', 'Err_basal_inferolateral', 'Err_basal_anterolateral', 'Err_mid_anterior', 'Err_mid_anteroseptal', 'Err_mid_inferoseptal', 'Err_mid_inferior', 'Err_mid_inferolateral', 'Err_mid_anterolateral', 'Err_apical_anterior', 'Err_apical_septal', 'Err_apical_inferior', 'Err_apical_lateral', 'Ecc_basal_anterior', 'Ecc_basal_anteroseptal', 'Ecc_basal_inferoseptal', 'Ecc_basal_inferior', 'Ecc_basal_inferolateral', 'Ecc_basal_anterolateral', 'Ecc_mid_anterior', 'Ecc_mid_anteroseptal', 'Ecc_mid_inferoseptal', 'Ecc_mid_inferior', 'Ecc_mid_inferolateral', 'Ecc_mid_anterolateral', 'Ecc_apical_anterior', 'Ecc_apical_septal', 'Ecc_apical_inferior', 'Ecc_apical_lateral', 'WTCI_basal_anterior', 'WTCI_basal_anteroseptal', 'WTCI_basal_inferoseptal', 'WTCI_basal_inferior', 'WTCI_basal_inferolateral

# spreadsheet for per-time-frame strain

In [23]:
def read_all_tf(folder, file_name, parameter_index, first_zero = True):
    parameter_list = []
    for tf in range(0,25):
        if tf == 0 and first_zero == True:
            parameter_list.append(0)
        else:
            file = np.load(os.path.join(folder, 'ED + ' + str(tf) , file_name), allow_pickle=True)
            if len(parameter_index) == 1:
                p = file[parameter_index[0]]
            else:
                p = file[parameter_index[0]][parameter_index[1]]
            parameter_list.append(np.round(p,3))
    return  np.asarray(parameter_list).reshape(-1) 


In [53]:
result = []

for i in range(0, 2):#patient_list.shape[0]):
    patient_id_num = patient_list['OurID'].iloc[i]
    patient_id = ff.XX_to_ID_00XX(patient_id_num)

    folder = os.path.join(main_path, 'results/strain', patient_id, 'per_time_frame')
    
    # do each parameter one by one (goint through all time frames)
    # global radial strain
    global_radial_strain_list = read_all_tf(folder, 'global_strain.npy', [0], first_zero = True)
    
    # global circumferential strain
    global_circumferential_strain_list = read_all_tf(folder, 'global_strain.npy', [1])

    # layer radial strain
    base_radial_strain_list = read_all_tf(folder, 'layer_strain.npy', [0,0])
    mid_radial_strain_list = read_all_tf(folder, 'layer_strain.npy', [0,1])
    apex_radial_strain_list = read_all_tf(folder, 'layer_strain.npy', [0,2])
    # layer circumferential strain
    base_circumferential_strain_list = read_all_tf(folder, 'layer_strain.npy', [1,0])
    mid_circumferential_strain_list = read_all_tf(folder, 'layer_strain.npy', [1,1])
    apex_circumferential_strain_list = read_all_tf(folder, 'layer_strain.npy', [1,2])
    
    # aha Err strain
    Err_basal_anterior_list = read_all_tf(folder, 'polar_strain.npy', [7,0])
    Err_basal_anteroseptal_list = read_all_tf(folder, 'polar_strain.npy', [7,1])
    Err_basal_inferoseptal_list = read_all_tf(folder, 'polar_strain.npy', [7,2])
    Err_basal_inferior_list = read_all_tf(folder, 'polar_strain.npy', [7,3])
    Err_basal_inferolateral_list = read_all_tf(folder, 'polar_strain.npy', [7,4])
    Err_basal_anterolateral_list = read_all_tf(folder, 'polar_strain.npy', [7,5])
    Err_mid_anterior_list = read_all_tf(folder, 'polar_strain.npy', [7,6])
    Err_mid_anteroseptal_list = read_all_tf(folder, 'polar_strain.npy', [7,7])
    Err_mid_inferoseptal_list = read_all_tf(folder, 'polar_strain.npy', [7,8])
    Err_mid_inferior_list = read_all_tf(folder, 'polar_strain.npy', [7,9])
    Err_mid_inferolateral_list = read_all_tf(folder, 'polar_strain.npy', [7,10])
    Err_mid_anterolateral_list = read_all_tf(folder, 'polar_strain.npy', [7,11])
    Err_apical_anterior_list = read_all_tf(folder, 'polar_strain.npy', [7,12])
    Err_apical_septal_list = read_all_tf(folder, 'polar_strain.npy', [7,13])
    Err_apical_inferior_list = read_all_tf(folder, 'polar_strain.npy', [7,14])
    Err_apical_lateral_list = read_all_tf(folder, 'polar_strain.npy', [7,15])

    # aha Ecc strain
    Ecc_basal_anterior_list = read_all_tf(folder, 'polar_strain.npy', [5,0])
    Ecc_basal_anteroseptal_list = read_all_tf(folder, 'polar_strain.npy', [5,1])
    Ecc_basal_inferoseptal_list = read_all_tf(folder, 'polar_strain.npy', [5,2])
    Ecc_basal_inferior_list = read_all_tf(folder, 'polar_strain.npy', [5,3])
    Ecc_basal_inferolateral_list = read_all_tf(folder, 'polar_strain.npy', [5,4])
    Ecc_basal_anterolateral_list = read_all_tf(folder, 'polar_strain.npy', [5,5])
    Ecc_mid_anterior_list = read_all_tf(folder, 'polar_strain.npy', [5,6])
    Ecc_mid_anteroseptal_list = read_all_tf(folder, 'polar_strain.npy', [5,7])
    Ecc_mid_inferoseptal_list = read_all_tf(folder, 'polar_strain.npy', [5,8])
    Ecc_mid_inferior_list = read_all_tf(folder, 'polar_strain.npy', [5,9])
    Ecc_mid_inferolateral_list = read_all_tf(folder, 'polar_strain.npy', [5,10])
    Ecc_mid_anterolateral_list = read_all_tf(folder, 'polar_strain.npy', [5,11])
    Ecc_apical_anterior_list = read_all_tf(folder, 'polar_strain.npy', [5,12])
    Ecc_apical_septal_list = read_all_tf(folder, 'polar_strain.npy', [5,13])
    Ecc_apical_inferior_list = read_all_tf(folder, 'polar_strain.npy', [5,14])
    Ecc_apical_lateral_list = read_all_tf(folder, 'polar_strain.npy', [5,15])

    # WTCI aha
    WTCI_basal_anterior_list = read_all_tf(folder, 'wtci.npy', [4,0])
    WTCI_basal_anteroseptal_list = read_all_tf(folder, 'wtci.npy', [4,1])
    WTCI_basal_inferoseptal_list = read_all_tf(folder, 'wtci.npy', [4,2])
    WTCI_basal_inferior_list = read_all_tf(folder, 'wtci.npy', [4,3])
    WTCI_basal_inferolateral_list = read_all_tf(folder, 'wtci.npy', [4,4])
    WTCI_basal_anterolateral_list = read_all_tf(folder, 'wtci.npy', [4,5])
    WTCI_mid_anterior_list = read_all_tf(folder, 'wtci.npy', [4,6])
    WTCI_mid_anteroseptal_list = read_all_tf(folder, 'wtci.npy', [4,7])
    WTCI_mid_inferoseptal_list = read_all_tf(folder, 'wtci.npy', [4,8])
    WTCI_mid_inferior_list = read_all_tf(folder, 'wtci.npy', [4,9])
    WTCI_mid_inferolateral_list = read_all_tf(folder, 'wtci.npy', [4,10])
    WTCI_mid_anterolateral_list = read_all_tf(folder, 'wtci.npy', [4,11])
    WTCI_apical_anterior_list = read_all_tf(folder, 'wtci.npy', [4,12])
    WTCI_apical_septal_list = read_all_tf(folder, 'wtci.npy', [4,13])
    WTCI_apical_inferior_list = read_all_tf(folder, 'wtci.npy', [4,14])
    WTCI_apical_lateral_list = read_all_tf(folder, 'wtci.npy', [4,15])


    # combine these results together into a array of (num_parameters,)
    a = np.concatenate((global_radial_strain_list, global_circumferential_strain_list, 
                        base_radial_strain_list, mid_radial_strain_list, apex_radial_strain_list, 
                        base_circumferential_strain_list, mid_circumferential_strain_list, apex_circumferential_strain_list, 
                        Err_basal_anterior_list, Err_basal_anteroseptal_list, Err_basal_inferoseptal_list, Err_basal_inferior_list, Err_basal_inferolateral_list, Err_basal_anterolateral_list, Err_mid_anterior_list, Err_mid_anteroseptal_list, Err_mid_inferoseptal_list, Err_mid_inferior_list, Err_mid_inferolateral_list, Err_mid_anterolateral_list, Err_apical_anterior_list, Err_apical_septal_list, Err_apical_inferior_list, Err_apical_lateral_list, 
                        Ecc_basal_anterior_list, Ecc_basal_anteroseptal_list, Ecc_basal_inferoseptal_list, Ecc_basal_inferior_list, Ecc_basal_inferolateral_list, Ecc_basal_anterolateral_list, Ecc_mid_anterior_list, Ecc_mid_anteroseptal_list, Ecc_mid_inferoseptal_list, Ecc_mid_inferior_list, Ecc_mid_inferolateral_list, Ecc_mid_anterolateral_list, Ecc_apical_anterior_list, Ecc_apical_septal_list, Ecc_apical_inferior_list, Ecc_apical_lateral_list, 
                        WTCI_basal_anterior_list, WTCI_basal_anteroseptal_list, WTCI_basal_inferoseptal_list, WTCI_basal_inferior_list, WTCI_basal_inferolateral_list, WTCI_basal_anterolateral_list, WTCI_mid_anterior_list, WTCI_mid_anteroseptal_list, WTCI_mid_inferoseptal_list, WTCI_mid_inferior_list, WTCI_mid_inferolateral_list, WTCI_mid_anterolateral_list, WTCI_apical_anterior_list, WTCI_apical_septal_list, WTCI_apical_inferior_list, WTCI_apical_lateral_list), 
                        axis=0)
    
    result.append(a)

In [66]:
# save into excel

result = np.asarray(result)
print(result.shape)

walls = ['basal_anterior', 'basal_anteroseptal', 'basal_inferoseptal', 'basal_inferior', 'basal_inferolateral', 'basal_anterolateral', 'mid_anterior', 'mid_anteroseptal', 'mid_inferoseptal', 'mid_inferior', 'mid_inferolateral', 'mid_anterolateral', 'apical_anterior', 'apical_septal', 'apical_inferior', 'apical_lateral']
parameter_list = ['Err_global', 'Ecc_global', 'Err_base', 'Err_mid', 'Err_apex', 'Ecc_base', 'Ecc_mid', 'Ecc_apex' ]
for n in ['Err', 'Ecc','WTCI']:
    for w in walls:
        parameter_list.append(n + '_' + w)

file_name = os.path.join(save_folder, 'strain_per_time_frame_collection.xlsx')

with pd.ExcelWriter(file_name) as writer:  
    for k in range(0, len(parameter_list)):
        r = result[:,25 * k : 25 * (k + 1)]
        r = r.tolist()
        
        column_list = []
        for tf in range(0,25):
            column_list.append(parameter_list[k] + '_ED+' + str(tf))
        df = pd.DataFrame(r, columns=column_list)
        df_info = patient_list[['OurID','Final label if count surgery at day 0', 'Final label if not count surgery at day 0', 'ZC label', 'ED', 'ES']]

        df = pd.concat([df_info, df], axis=1)
        df.to_excel(writer, sheet_name=parameter_list[k], index=False)



(2, 1400)


# overlay images + segmentation + strain map

In [6]:
save_overlay_folder = os.path.join(save_folder, 'overlay_images')

In [8]:
manual_seg_list = [True, False]

for patient_index in range(0, patient_list.shape[0]):
    for manual_seg in manual_seg_list:
        patient_id_num = patient_list['OurID'].iloc[patient_index]
        patient_id = ff.XX_to_ID_00XX(patient_id_num)
        patient_row = raw_spreadsheet.loc[raw_spreadsheet['OurID'] == patient_id_num]
        ED = patient_row['ED'].values[0].astype(int)
        ES = patient_row['ES'].values[0].astype(int)
        print(patient_id, ED,ES)

        save_overlay_folder_case = os.path.join(save_overlay_folder, patient_id); ff.make_folder([save_overlay_folder_case])

        # second: load img and seg
        patient_img_folder = os.path.join(data_path, 'nii_img', patient_id)
        patient_seg_folder = os.path.join(data_path, 'nii_manual_seg', patient_id)

        V_nifti_ED = nb.load(os.path.join(patient_img_folder, 'Org3D_frame' + str(ED) + '.nii.gz'))
        V_nifti_ES = nb.load(os.path.join(patient_img_folder, 'Org3D_frame' + str(ES) + '.nii.gz'))

        if manual_seg == True:
            M_nifti_ED = nb.load(os.path.join(patient_seg_folder, 'SAX_ED_seg.nii.gz'))
            M_nifti_ES = nb.load(os.path.join(patient_seg_folder, 'SAX_ES_seg.nii.gz'))
        else:
            M_nifti_ED= nb.load(os.path.join(main_path,'results/fine_tune_carson/seg/' ,patient_id,'pred_seg_frame' + str(ED) + '.nii.gz'))
            M_nifti_ES = nb.load(os.path.join(main_path,'results/fine_tune_carson/seg/' ,patient_id,'pred_seg_frame' + str(ES) + '.nii.gz')) 


        M_ED = np.round(M_nifti_ED.get_fdata()).astype(int)
        new_M_ED = np.zeros_like(M_ED)
        new_M_ED[M_ED==1] = 3
        new_M_ED[M_ED==2] = 2
        M_nifti_ED = nb.Nifti1Image(new_M_ED, affine=M_nifti_ED.affine, header=M_nifti_ED.header)

        M_ES = np.round(M_nifti_ES.get_fdata()).astype(int)
        new_M_ES = np.zeros_like(M_ES)
        new_M_ES[M_ES==1] = 3
        new_M_ES[M_ES==2] = 2
        M_nifti_ES = nb.Nifti1Image(new_M_ES, affine=M_nifti_ES.affine, header=M_nifti_ES.header)

        V_nifti = nb.funcs.concat_images((V_nifti_ED, V_nifti_ES))
        M_nifti = nb.funcs.concat_images((M_nifti_ED, M_nifti_ES))

        V_nifti = resample_nifti(V_nifti, order=1, in_plane_resolution_mm=1.25, number_of_slices=16)
        M_nifti = resample_nifti(M_nifti, order=0, in_plane_resolution_mm=1.25, number_of_slices=16)

        center = center_of_mass(M_nifti.get_fdata()[:,:,:,0]==2) # RV = 1, Myocardium = 2, LV = 3
        V = _roll2center_crop(x=V_nifti.get_fdata(), center=center)
        M = _roll2center_crop(x=M_nifti.get_fdata(), center=center)

        # 
        V = ff.normalize_image(V)
        nx, ny, nz, nt = V.shape
        V_0 =  np.repeat(V[:,:,:,:1], nt-1, axis=-1)
        V_t =  V[:,:,:,1:]
        V_0 = np.transpose(V_0, (3,0,1,2))
        V_t = np.transpose(V_t, (3,0,1,2))


        y_t = nb.load(os.path.join(main_path, 'results/trained/mvf', patient_id, 'mvf_ED_ES.nii.gz')).get_fdata()
        y_t = y_t[None,...]

        mask_end_diastole = M[..., 0]
        mask_end_systole = M[..., 1]

        # strain
        strain = np.load(os.path.join(main_path,'results/strain', patient_id, 'raw_strain.npy'), allow_pickle=True)[0]

        # slice_info
        slice_info = np.load(os.path.join(main_path,'results/strain', patient_id, 'slice_info.npy'), allow_pickle=True)
        slice_effective = slice_info[0]
        start_slice_name = slice_info[3]
        if start_slice_name == 'apex':
            # revert slice_effective
            slice_effective = slice_effective[::-1]


        # plot
        # Define the number of rows and columns for subplots
        num_rows, num_cols = len(slice_effective), 4

        # Create a figure and axes for subplots
        fig, axs = plt.subplots(num_rows, num_cols, figsize=(20,5 * len(slice_effective)))  # Adjust figsize as needed

        # Populate the subplots with content (example: displaying subplot index)
        for i in range(num_rows):
            for j in range(num_cols):
                if j == 0:
                    axs[i,j].imshow(V_0[0, :, :, slice_effective[i]], cmap='gray')
                    axs[i,j].set_ylabel('slice_' + str(i+1), fontsize=28)
                    axs[i, j].tick_params(axis='y', which='both', left=False, right=False, labelleft=False)  # Turn off y-axis ticks
                    axs[i, j].get_xaxis().set_visible(False)  # Turn off x-axis including ticks and labels
                    if i == 0:
                        axs[i,j].set_title('ED_img', fontsize=30)
                elif j == 1:
                    axs[i,j].imshow(V_t[0, :, :, slice_effective[i]], cmap='gray'); axs[i,j].axis('off')
                    if i == 0:
                        axs[i,j].set_title('ES_img', fontsize=30)
                elif j == 2:
                    imgplot = axs[i,j].imshow(strain.Err[:, :, slice_effective[i]], cmap='RdBu'); axs[i,j].axis('off')
                    if i == 0:
                        axs[i,j].set_title('Err', fontsize=30)
                else:
                    strain_map = np.copy(strain.Err[:, :, slice_effective[i]])
                    seg = mask_end_diastole[:,:, slice_effective[i]]
                    strain_map[seg == 2] = np.max(strain_map)
                    imgplot = axs[i,j].imshow(strain_map, cmap='RdBu'); axs[i,j].axis('off')
                    if i == 0:
                        axs[i,j].set_title('Err + Myo-seg', fontsize=30)

        # Adjust layout and display the figure
        plt.tight_layout()
        plt.subplots_adjust(wspace=0, hspace=0)

        if manual_seg == True:
            filename = 'ED_ES_radial_strain_with_manual_segmentation.png'
        else:
            filename = 'ED_ES_radial_strain_with_pred_segmentation.png'
        plt.savefig(os.path.join(save_overlay_folder_case,filename), format='png', bbox_inches='tight')
        plt.close()



ID_0692 1 8
ID_0692 1 8
ID_1207 1 6
ID_1207 1 6
ID_0085 1 10
ID_0085 1 10
ID_1208 1 8
ID_1208 1 8
ID_0685 1 8
ID_0685 1 8
ID_0483 1 8
ID_0483 1 8
ID_0682 1 8
ID_0682 1 8
ID_0290 1 9
ID_0290 1 9
ID_0954 1 7
ID_0954 1 7
ID_0291 1 8
ID_0291 1 8
ID_0953 1 8
ID_0953 1 8
ID_0678 1 8
ID_0678 1 8
ID_0287 1 10
ID_0287 1 10
ID_1183 1 9
ID_1183 1 9
ID_1180 1 10
ID_1180 1 10
ID_1181 1 9
ID_1181 1 9
ID_0951 1 7
ID_0951 1 7
ID_0080 1 7
ID_0080 1 7
ID_1172 1 8
ID_1172 1 8
ID_0949 1 7
ID_0949 1 7
ID_1405 1 10
ID_1405 1 10
ID_1177 1 7
ID_1177 1 7
ID_0284 1 9
ID_0284 1 9
ID_1175 1 8
ID_1175 1 8
ID_1361 1 5
ID_1361 1 5
ID_0468 1 13
ID_0468 1 13
ID_0672 3 10
ID_0672 3 10
ID_1057 1 9
ID_1057 1 9
ID_0671 1 8
ID_0671 1 8
ID_1151 1 8
ID_1151 1 8
ID_0016 1 7
ID_0016 1 7
ID_1163 1 9
ID_1163 1 9
ID_0824 1 9
ID_0824 1 9
ID_0280 1 8
ID_0280 1 8
ID_0078 1 7
ID_0078 1 7
ID_1354 1 9
ID_1354 1 9
ID_1141 1 9
ID_1141 1 9
ID_1353 1 8
ID_1353 1 8
ID_1130 1 10
ID_1130 1 10
ID_1124 1 7
ID_1124 1 7
ID_0940 1 12
ID_0940 1 12
